In [15]:
import openai
import pandas as pd
import numpy as np
import re

In [20]:
df = pd.read_excel("../2차결과.xlsx") 
df = df.drop(["review"], axis=1)

df['학생명'] = df['학생명'].apply(lambda x: re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]', '', x))

print(df)

      학생명           제출 시간       문제 이름 정답 여부 오류 분류  \
0     강지민  22.11.01 22:08  10보다작을까?-1    AC   NaN   
1     권현욱  21.12.26 10:24  10보다작을까?-1    AC   NaN   
2     김강범  22.07.30 18:11  10보다작을까?-1    AC   NaN   
3     김동우  22.11.05 10:41  10보다작을까?-1    AC   NaN   
4     김민하  21.12.18 11:24  10보다작을까?-1    AC   NaN   
...   ...             ...         ...   ...   ...   
1466  허윤구  21.03.07 11:11        사칙연산    WA    -1   
1467  허윤구  21.03.07 11:10        사칙연산    WA    -1   
1468  허윤구  21.03.07 11:07        사칙연산    WA     0   
1469  허윤구  21.03.07 11:07        사칙연산    WA     2   
1470  홍동인  21.03.07 11:06        사칙연산    AC   NaN   

                                                     코드  \
0     #include <cstdio>\nint main(){\n    int a;\n  ...   
1     #include<stdio.h>\nint main()\n{\nint a;\nscan...   
2     #include <cstdio>\n\nint main()\n{\n    int a;...   
3     #include<cstdio>\nint main(){\n    int a;\n   ...   
4     #include<cstdio>\nint main(){\nint a;\nscanf("...   
...      

In [21]:
df.to_excel('../3차결과.xlsx', index=False, engine='openpyxl')

이전 시도

In [7]:
df['제출 시간'] = pd.to_datetime(df['제출 시간'], format='%y.%m.%d %H:%M')

df['제출 시간'] = df['제출 시간'].dt.strftime('%y.%m.%d')

df = df.sort_values('제출 시간')

print(df.loc[df["학생명"]=="송지원", ["학생명","제출 시간"]].to_string(index=False))

학생명    제출 시간
송지원 20.05.16
송지원 20.05.18
송지원 20.05.18
송지원 20.05.19
송지원 20.05.20
송지원 20.05.21
송지원 20.05.21
송지원 20.05.21
송지원 20.05.21
송지원 20.05.23
송지원 20.05.23
송지원 20.05.23
송지원 20.05.23
송지원 20.05.27
송지원 20.05.27
송지원 20.05.29


In [8]:
# 학생별로 회차 계산을 위한 변수 초기화
prev_date_by_student = {}
counter_by_student = {}

# 회차 계산 함수 정의
def calculate_round(row):
    student = row["학생명"]
    date = row["제출 시간"]
    
    if student not in prev_date_by_student:
        prev_date_by_student[student] = None
        counter_by_student[student] = 0
    
    prev_date = prev_date_by_student[student]
    counter = counter_by_student[student]
    
    if date != prev_date:
        counter += 1
        prev_date = date
    
    prev_date_by_student[student] = prev_date
    counter_by_student[student] = counter
    
    return counter

# 회차 계산하여 "회차" 열 추가
df["일자"] = df.apply(calculate_round, axis=1)

In [9]:
# 결과 출력
print(df.loc[:, ["학생명","제출 시간", "일자"]].to_string(index=False))

 학생명    제출 시간  일자
 송지원 20.05.16   1
 송지원 20.05.18   2
 송지원 20.05.18   2
 송지원 20.05.19   3
 송지원 20.05.20   4
 송지원 20.05.21   5
 송지원 20.05.21   5
 송지원 20.05.21   5
 송지원 20.05.21   5
 송지원 20.05.23   6
 송지원 20.05.23   6
 송지원 20.05.23   6
 송지원 20.05.23   6
 송지원 20.05.27   7
 송지원 20.05.27   7
 송지원 20.05.29   8
 이연준 20.10.25   1
 이연준 20.10.25   1
 이연준 20.10.25   1
 이연준 20.10.25   1
 이연준 20.10.25   1
 이연준 20.10.25   1
 이연준 20.10.25   1
 이연준 20.10.25   1
 이연준 20.10.25   1
 이연준 20.10.25   1
 이연준 20.10.25   1
 이연준 20.10.25   1
 이연준 20.10.25   1
 이연준 20.10.25   1
 이연준 20.10.28   2
 이연준 20.10.29   3
 이연준 20.10.29   3
 이연준 20.10.29   3
 이연준 20.10.30   4
 이연준 20.10.30   4
 이연준 20.10.30   4
 이연준 20.10.30   4
 이연준 20.10.30   4
 이연준 20.11.01   5
 이연준 20.11.01   5
 이연준 20.11.01   5
 이연준 20.11.01   5
 이연준 20.11.02   6
 이연준 20.11.02   6
 이연준 20.11.02   6
 이연준 20.11.02   6
 이연준 20.11.02   6
 이연준 20.11.02   6
 김승희 20.11.12   1
 김승희 20.11.12   1
 김승희 20.11.12   1
 김승희 20.11.13   2
 김승희 20.11.13   2
 김승희 20.11

In [11]:
# 중복된 문제 이름 제외하고 나머지 제거
new_df = df.drop_duplicates(subset=['학생명', '제출 시간'], keep='first')

In [12]:
# 문제 순서별 학생 수 계산
student_counts = new_df["일자"].value_counts().sort_index()

student_counts

일자
1    53
2    50
3    44
4    30
5    21
6    15
7    11
8     6
9     1
Name: count, dtype: int64

In [110]:
df.to_excel('../3차결과.xlsx', index=False, engine='openpyxl')